# Web Services

__Description & purpose__:  This notebook introduces the webservices available through the EODH platform and API endpoints.  

__Author(s)__: Alastair Graham

__Date created__: 2024-12-05

__Date last modified__: 2024-12-17

__Licence__: This notebook is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  The code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

[WIP Icon Attribution]("https://www.flaticon.com/free-icons/work-in-progress")

# Introduction

OGC API refers to a standardised interface defined by the [Open Geospatial Consortium (OGC)(https://ogcapi.ogc.org/) for accessing and sharing geospatial data over the web. These APIs are built on modern web standards, such as HTTP and RESTful principles, to facilitate seamless, interoperable, and efficient access to spatial data and services. By providing a consistent and easy-to-use framework, OGC APIs allow developers, researchers, and organisations to query, retrieve, and analyze geospatial data without being locked into proprietary systems. This is particularly important as it supports interoperability between different platforms, tools, and organizations, enabling collaboration and innovation. OGC APIs play a critical role in breaking down data silos and ensuring that location-based data can be shared across applications.

## Records
OGC API - Records is designed for discovering and accessing metadata about geospatial resources. It enables users to search, retrieve, and manage metadata records, such as datasets, services, and other resources. The standard supports flexible query capabilities, filtering, and sorting to help users discover relevant geospatial content efficiently. By adhering to web and open standards, OGC API - Records ensures interoperability across platforms, making it easier for organizations to share and access geospatial metadata in a consistent, machine-readable format. This API service is used on the backend of EODH.

![](../img/wip.png){width=50px} 

**To be added**

* List of web services and what they do
* Example code on how to access them and visualise the related data
* How to use the services in QGIS 